Endprojekt GIS Analyse - Location Analysis for a new Stadium in Graz

In [8]:
# Data manipulation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Geospatial
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Polygon
import osmnx as ox

# Raster processing
import os
import pyproj
# optinal code fix with PROJ_LIB if pyproj doesnt work properly
os.environ["PROJ_LIB"] = pyproj.datadir.get_data_dir()
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.crs import CRS
from rasterio.features import shapes
from shapely.geometry import shape


Data Preparation
===============================================================================     

Getting district boundaries of Graz (District boundaries (Overpass Turbo (n.d.). Overpass API web interface. https://overpass-turbo.eu/ (Accessed December 2, 2025))) <br>
and <br>
population statistic (Stadt Graz (2025). Zahlen + Fakten: Bevölkerung, Bezirke, Wirtschaft, Geografie. https://www.graz.at/cms/beitrag/10034466/7772565/Zahlen_Fakten_Bevoelkerung_Bezirke_Wirtschaft.html (Accessed December 2, 2025))

In [9]:
#loading GeoJson Districts of Graz
gdf_districts = gpd.read_file("data/district_graz.geojson")
gdf_districts = gdf_districts[["name", "geometry"]].copy() #extract the columns needed

#loading csv with pop data of graz 
df_popgraz = pd.read_csv("data/Bevoelkerung Graz.csv", encoding="latin1", sep=";")

#merge the two datasets
gdf_districts_popGraz = gdf_districts.merge(df_popgraz, left_on="name", right_on="Bezirk")

Getting hard exklusion layers from OpenStreetMap via OSMnx, set the CRS to 32633 (UTM zone 33N) and buffer some of them:<br>
- water bodies buffer (10m)<br>
- parks/green areas <br>
- transport infrastructure <br>
- buildings (buffer 250m for hospitals, kingardens, cemeteries)

In [10]:
PLACE_NAME:str = "Graz, Austria"
#TARGET_CRS = "EPSG:32633"  # UTM zone 33N

In [11]:
tags_water = {
    "natural": ["water"],
    "waterway": ["river", "stream", "canal", "ditch"],
    "landuse": ["reservoir"],
    "water": ["lake", "river", "pond", "basin"]
}
gdf_water = ox.features_from_place(
    PLACE_NAME,
    tags=tags_water
)
gdf_water = gdf_water[["geometry"]].copy()
gdf_water["category"] = "water"

# Reproject
gdf_water = gdf_water.to_crs(epsg=32633)

# 10 meter buffer
gdf_water_Buffer10m = gdf_water.copy()
gdf_water_Buffer10m["geometry"] = gdf_water_Buffer10m.buffer(10)
gdf_water_Buffer10m.head()

geometry category
element  id                                                                 
relation 1306807  POLYGON ((532504.7 5213425.005, 532504.197 521...    water
         2325656  POLYGON ((536875.338 5207617.465, 536875.714 5...    water
         3403202  POLYGON ((536217.762 5210905.412, 536219.526 5...    water
         3562966  POLYGON ((533585.365 5209757.598, 533585.028 5...    water
         3587716  POLYGON ((535239.839 5206807.914, 535240.715 5...    water

In [12]:
#Excklusion Parks
tags_parks = {
    "leisure": [
        "park", "garden", "playground", "recreation_ground"
    ],
    "boundary": [
        "protected_area"
    ]
}
gdf_parks = ox.features_from_place(
    PLACE_NAME,
    tags=tags_parks
)

gdf_parks = gdf_parks[["geometry"]].copy()
gdf_parks["category"] = "green areas"

# Reproject
gdf_parks = gdf_parks.to_crs(epsg=32633)

# 10 meter buffer
gdf_parks_Buffer10m = gdf_parks.copy()
gdf_parks_Buffer10m["geometry"] = gdf_parks_Buffer10m.buffer(10)
gdf_parks_Buffer10m.head(5)

geometry  \
element id                                                              
node    859421763   POLYGON ((529248.583 5215245.914, 529248.535 5...   
        940994980   POLYGON ((532485.494 5215082.215, 532485.446 5...   
        1053837597  POLYGON ((534672.168 5213142.265, 534672.12 52...   
        1265834876  POLYGON ((532016.172 5209443.055, 532016.124 5...   
        1412653685  POLYGON ((532000.365 5216204.807, 532000.317 5...   

                       category  
element id                       
node    859421763   green areas  
        940994980   green areas  
        1053837597  green areas  
        1265834876  green areas  
        1412653685  green areas

In [13]:
#Exclusion Transport Areas
tags_transport = {
    "highway": [
        "motorway", "trunk", "primary", "secondary", "tertiary",
        "motorway_link", "trunk_link", "primary_link", "secondary_link"
    ],
    "railway": [
        "rail", "tram", "light_rail", "subway"
    ]
}

gdf_transport = ox.features_from_place(
    PLACE_NAME,
    tags=tags_transport
)

gdf_transport = gdf_transport[["geometry"]].copy()
gdf_transport["category"] = "transport infrastructure"

# Reproject
gdf_transport = gdf_transport.to_crs(epsg=32633)

# 5 meter buffer
gdf_transport_Buffer5m = gdf_transport.copy()
gdf_transport_Buffer5m["geometry"] = gdf_transport_Buffer5m.buffer(5)
gdf_transport_Buffer5m.head(5)

geometry  \
element id                                                           
way     3987860  POLYGON ((533457.435 5211123.551, 533473.569 5...   
        3991673  POLYGON ((533270.339 5213182.195, 533270.367 5...   
        3991674  POLYGON ((534345.324 5211854.726, 534345.549 5...   
        3992913  POLYGON ((533607.999 5212406.65, 533607.991 52...   
        3992916  POLYGON ((536360.953 5209709.825, 536364.257 5...   

                                 category  
element id                                 
way     3987860  transport infrastructure  
        3991673  transport infrastructure  
        3991674  transport infrastructure  
        3992913  transport infrastructure  
        3992916  transport infrastructure

In [14]:
#Exclusion Buildings
tags_buildings_hard = {
    "building": True
}

gdf_buildings = ox.features_from_place(
    PLACE_NAME,
    tags=tags_buildings_hard
)

gdf_buildings = gdf_buildings[["geometry"]].copy()
gdf_buildings["category"] = "buildings_hard"

# Reproject
gdf_buildings = gdf_buildings.to_crs(epsg=32633)

In [15]:
#Cemeteries with 250m Buffer
tags_cemetery = {
    "landuse": ["cemetery"],
    "amenity": ["grave_yard"]
}

gdf_cemetery = ox.features_from_place(
    PLACE_NAME,
    tags=tags_cemetery
)

gdf_cemetery = gdf_cemetery[["geometry"]].copy()
gdf_cemetery["category"] = "cemetery"

gdf_cemetery = gdf_cemetery.to_crs(epsg=32633)
gdf_cemetery.head(5)

geometry  \
element  id                                                              
node     1447016305                      POINT (535889.182 5209509.97)   
relation 1252339     POLYGON ((531967.661 5210631.508, 531988.86 52...   
         1252342     POLYGON ((532066.233 5210275.077, 532053.994 5...   
         2694625     MULTIPOLYGON (((531233.48 5218089.831, 531255....   
         17946151    POLYGON ((534965.319 5212317.125, 534995.58 52...   

                     category  
element  id                    
node     1447016305  cemetery  
relation 1252339     cemetery  
         1252342     cemetery  
         2694625     cemetery  
         17946151    cemetery

Merge to one geodataframe and dissolve to one geometry

In [16]:
gdf_exclusion_areas = pd.concat([
    gdf_water_Buffer10m,
    gdf_parks_Buffer10m,
    gdf_transport_Buffer5m,
    gdf_buildings,
    gdf_cemetery,
], ignore_index=True)

In [17]:
gdf_exclusion_areas.head(5)

,geometry,category
0,"POLYGON ((532504.7 5213425.005, 532504.197 521...",water
1,"POLYGON ((536875.338 5207617.465, 536875.714 5...",water
2,"POLYGON ((536217.762 5210905.412, 536219.526 5...",water
3,"POLYGON ((533585.365 5209757.598, 533585.028 5...",water
4,"POLYGON ((535239.839 5206807.914, 535240.715 5...",water


In [18]:
gdf_exclusion_areas.geometry.geom_type.value_counts()


Polygon         70581
Point              50
MultiPolygon       18
Name: count, dtype: int64

In [19]:
gdf_exclusion_areas = gdf_exclusion_areas[
    gdf_exclusion_areas.geometry.geom_type != "Point"
]
gdf_exclusion_areas = gdf_exclusion_areas.explode(
    ignore_index=True
)
gdf_exclusion_areas.geometry.geom_type.value_counts()

Polygon    70631
Name: count, dtype: int64

Reprojecting DEM from WGS84 to UTM33N and calculate slope 

In [20]:
dem_in = "data/DEM30_Graz.tif"          # original DEM
dem_utm = "data/DEM30_Graz_utm33.tif"   # output DEM (UTM33N)

with rasterio.open(dem_in) as src:
    src_crs = src.crs if src.crs is not None else CRS.from_epsg(4326)
    dst_crs = CRS.from_epsg(32633)  # UTM Zone 33N

    transform, width, height = calculate_default_transform(
        src_crs, dst_crs, src.width, src.height, *src.bounds
    )

    profile = src.profile.copy()
    profile.update({
        "crs": dst_crs,
        "transform": transform,
        "width": width,
        "height": height,
        "nodata": src.nodata if src.nodata is not None else -9999
    })

    with rasterio.open(dem_utm, "w", **profile) as dst:
        dst_array = np.full((height, width), profile["nodata"], dtype=profile["dtype"])
        dst.write(dst_array, 1)

        reproject(
            source=rasterio.band(src, 1),
            destination=rasterio.band(dst, 1),
            src_transform=src.transform,
            src_crs=src_crs,
            dst_transform=transform,
            dst_crs=dst_crs,
            resampling=Resampling.bilinear,
            src_nodata=src.nodata,
            dst_nodata=profile["nodata"]
        )

In [21]:
dem_path = "data/DEM30_Graz_utm33.tif"
slope_path = "data/dem_slope.tif"

with rasterio.open(dem_path) as src:
    dem = src.read(1, masked=True).astype("float64")
    profile = src.profile.copy()
    transform = src.transform

xres = transform.a
yres = abs(transform.e)

dzdy, dzdx = np.gradient(dem.filled(np.nan), yres, xres)

slope_rad = np.arctan(np.sqrt(dzdx**2 + dzdy**2))
slope_deg = np.degrees(slope_rad)

nodata_out = -9999.0
slope_out = np.where(np.isnan(slope_deg), nodata_out, slope_deg).astype(np.float32)

profile.update(dtype=rasterio.float32, count=1, nodata=nodata_out)

with rasterio.open(slope_path, "w", **profile) as dst:
    dst.write(slope_out, 1)


In [23]:
slope_path = "data/dem_slope.tif"
out_path   = "data/slope_filtered.shp"

threshold   = 5.0     # Grad
min_area_m2 = 500.0   # Mindestfläche

with rasterio.open(slope_path) as src:
    slope = src.read(1)
    transform = src.transform
    crs = src.crs
    nodata = src.nodata

mask = slope > threshold
if nodata is not None:
    mask = mask & (slope != nodata)
mask = mask & np.isfinite(slope)

value_raster = mask.astype(np.uint8)

geoms = []
for geom, value in shapes(value_raster, mask=mask, transform=transform):
    if value == 1:
        geoms.append(shape(geom))

gdf_slope = gpd.GeoDataFrame(geometry=geoms, crs=crs)

gdf_slope["diss"] = 1
gdf_slope = gdf_slope.dissolve(by="diss")
gdf_slope = gdf_slope.explode(index_parts=False).reset_index(drop=True)

gdf_slope["area_m2"] = gdf_slope.area
gdf_slope = gdf_slope[gdf_slope["area_m2"] >= min_area_m2]

gdf_slope["slope_gt"] = threshold
gdf_slope = gdf_slope[["slope_gt", "area_m2", "geometry"]]
gdf_slope.to_file(out_path, driver="ESRI Shapefile")

print(f"{len(gdf_slope)} Polygone gespeichert in {out_path}")

4307 Polygone gespeichert in data/slope_filtered.shp


c:\Users\cleam\Documents\UNI\WS2025\Gat2\venv-gat2\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Value 166892829.33209983 of field area_m2 of feature 3466 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(


# Weighting and Scoring

medium weight criteria when not near by kindergarten, hospital, cemetery and churches <br>
high weight criteria when near by transport infrastructure <br>

In [ ]:
#filter kindergartens
tags_kindergarten = {
    "amenity": ["kindergarten"]
}

gdf_kindergarten = ox.features_from_place(
    PLACE_NAME,
    tags=tags_kindergarten
)

gdf_kindergarten = gdf_kindergarten[["geometry"]].copy()
gdf_kindergarten["category"] = "kindergarten"

gdf_kindergarten = gdf_kindergarten.to_crs(epsg=32633)
gdf_kindergarten.head(5)


In [ ]:
#filter hospitals
tags_hospital = {
    "amenity": ["hospital"]
}

gdf_hospital = ox.features_from_place(
    PLACE_NAME,
    tags=tags_hospital
)

gdf_hospital = gdf_hospital[["geometry"]].copy()
gdf_hospital["category"] = "hospital"

gdf_hospital = gdf_hospital.to_crs(epsg=32633)
gdf_hospital.head(5)

In [ ]:
#filter churches

In [ ]:
#filter cemeteries 
tags_cemetery = {
    "amenity": ["cemetery"]
}

gdf_cemetery = ox.features_from_place(
    PLACE_NAME,
    tags=tags_cemetery
)

gdf_cemetery = gdf_cemetery[["geometry"]].copy()
gdf_cemetery["category"] = "cemetery"

gdf_cemetery = gdf_cemetery.to_crs(epsg=32633)
gdf_cemetery.head(5)

In [ ]:
# Public transport stops in Graz
tags_stops = {
    "highway": ["bus_stop"],
    "public_transport": ["platform", "stop_position"],
    "railway": ["tram_stop"]
}

gdf_stops = ox.features_from_place(
    PLACE_NAME,
    tags=tags_stops
)

# Nur Geometrie behalten
gdf_stops = gdf_stops[["geometry"]].copy()

# Kategorie setzen
gdf_stops["category"] = "public_transport_stop"

In [ ]:
#residental areas